## Synthesis

## Synthesis

In [3]:
def minimize_fds(fds):
    # Step 1: Simplify RHS of each FD
    print('Step 1:')
    simplified_fds = []
    for lhs, rhs in fds:
        for r in rhs:
            simplified_fds.append((frozenset(lhs), frozenset([r])))
    print('Simplify right side of each FD: ',simplified_fds)
    print('\n')

    # Step 2: Simplify LHS of each FD
    print('Step 2:')
    for i in range(len(simplified_fds)):
        lhs, rhs = simplified_fds[i]
        for attr in lhs:
            temp_lhs = lhs - frozenset([attr])
            closure = compute_closure(temp_lhs, simplified_fds)
            if rhs.issubset(closure):
                lhs = temp_lhs
        simplified_fds[i] = (lhs, rhs)
    print('Simplify left side of each FD: ', simplified_fds)
    print('\n')

    # Step 3: Remove extraneous dependencies
    print('Step 3:')
    minimized_fds = simplified_fds.copy()
    for fd in simplified_fds:
        minimized_fds.remove(fd)
        if not fd[1].issubset(compute_closure(fd[0], minimized_fds)):
            minimized_fds.append(fd)
    print('Minimized extra dependencies: ',minimized_fds)
    print('\n')

    # Step 4: Combine FDs with same LHS
    combined_fds = {}
    for lhs, rhs in minimized_fds:
        lhs = tuple(lhs)  # Convert frozenset to tuple
        if lhs in combined_fds:
            combined_fds[lhs] = combined_fds[lhs].union(rhs)
        else:
            combined_fds[lhs] = rhs
    return [(set(lhs), combined_fds[lhs]) for lhs in combined_fds]

def compute_closure(attributes, fds):
    closure = set(attributes)
    while True:
        new_attributes = closure.copy()
        for lhs, rhs in fds:
            if set(lhs).issubset(closure) and not rhs.issubset(closure):
                new_attributes = new_attributes.union(rhs)
        if new_attributes == closure:
            break
        closure = new_attributes
    return closure

# Example usage
functional_dependencies = [
    ({"A", "B"}, {"C", "D", "E"}),
    ({"A", "C"}, {"B", "D", "E"}),
]
minimal_cover = minimize_fds(functional_dependencies)
print(minimal_cover)



Step 1:
Simplify right side of each FD:  [(frozenset({'B', 'A'}), frozenset({'C'})), (frozenset({'B', 'A'}), frozenset({'E'})), (frozenset({'B', 'A'}), frozenset({'D'})), (frozenset({'C', 'A'}), frozenset({'B'})), (frozenset({'C', 'A'}), frozenset({'E'})), (frozenset({'C', 'A'}), frozenset({'D'}))]


Step 2:
Simplify left side of each FD:  [(frozenset({'B', 'A'}), frozenset({'C'})), (frozenset({'B', 'A'}), frozenset({'E'})), (frozenset({'B', 'A'}), frozenset({'D'})), (frozenset({'C', 'A'}), frozenset({'B'})), (frozenset({'C', 'A'}), frozenset({'E'})), (frozenset({'C', 'A'}), frozenset({'D'}))]


Step 3:
Minimized extra dependencies:  [(frozenset({'B', 'A'}), frozenset({'C'})), (frozenset({'C', 'A'}), frozenset({'B'})), (frozenset({'C', 'A'}), frozenset({'E'})), (frozenset({'C', 'A'}), frozenset({'D'}))]


[({'B', 'A'}, frozenset({'C'})), ({'C', 'A'}, frozenset({'E', 'B', 'D'}))]


## Candidate Key and Attribute Closures

In [1]:
import itertools

def find_candidate_keys(attributes, fds):
    # Find all subsets of the attribute set
    all_subsets = find_subsets(attributes)

    # Compute the closure of each subset and check if it covers all attributes
    candidate_keys = []
    for subset in all_subsets:
        closure = compute_closure(subset, fds)
        if closure == attributes:
            # Check if it's a minimal superkey
            if all(not set(key).issubset(subset) for key in candidate_keys):
                # Remove any supersets of this key
                candidate_keys = [key for key in candidate_keys if not subset.issubset(set(key))]
                candidate_keys.append(subset)
    return candidate_keys

def find_subsets(s):
    # Returns all subsets of a set s
    subsets = []
    for i in range(1, len(s) + 1):
        subsets.extend(map(set, itertools.combinations(s, i)))
    return subsets

def compute_closure(attributes, fds):
    # Compute the closure of a set of attributes under the functional dependencies
    closure = set(attributes)
    while True:
        new_attributes = closure.copy()
        for lhs, rhs in fds:
            if set(lhs).issubset(closure) and not rhs.issubset(closure):
                new_attributes = new_attributes.union(rhs)
        if new_attributes == closure:
            break
        closure = new_attributes
    return closure

def print_attribute_closures(attributes, fds):
    all_subsets = find_subsets(attributes)
    # Sort the subsets: first by length, then alphabetically
    sorted_subsets = sorted(all_subsets, key=lambda subset: (len(subset), ''.join(sorted(subset))))
    for subset in sorted_subsets:
        closure = compute_closure(subset, fds)
        print(f"Closure of {sorted(subset)}: {closure}")

# Example usage
attributes = {"A", "B", "C", "D", "E", "G"}  # Replace with attributes of your relation
functional_dependencies = [
    ({"C"}, {"D"}),
    ({"D", "E"}, {"C"}),
    ({"G"}, {"B", "D"}),
    ({"B"}, {"A", "G"}),
]

candidate_keys = find_candidate_keys(attributes, functional_dependencies)
print("Candidate Keys:", candidate_keys)

# Print closures of all attributes
print("\nAttribute Closures:")
print_attribute_closures(attributes, functional_dependencies)



Candidate Keys: [{'G', 'E'}, {'E', 'B'}]

Attribute Closures:
Closure of ['A']: {'A'}
Closure of ['B']: {'G', 'D', 'A', 'B'}
Closure of ['C']: {'C', 'D'}
Closure of ['D']: {'D'}
Closure of ['E']: {'E'}
Closure of ['G']: {'G', 'D', 'A', 'B'}
Closure of ['A', 'B']: {'G', 'D', 'A', 'B'}
Closure of ['A', 'C']: {'C', 'D', 'A'}
Closure of ['A', 'D']: {'D', 'A'}
Closure of ['A', 'E']: {'E', 'A'}
Closure of ['A', 'G']: {'G', 'D', 'B', 'A'}
Closure of ['B', 'C']: {'G', 'D', 'A', 'C', 'B'}
Closure of ['B', 'D']: {'G', 'D', 'B', 'A'}
Closure of ['B', 'E']: {'G', 'D', 'A', 'C', 'E', 'B'}
Closure of ['B', 'G']: {'G', 'D', 'A', 'B'}
Closure of ['C', 'D']: {'C', 'D'}
Closure of ['C', 'E']: {'C', 'E', 'D'}
Closure of ['C', 'G']: {'G', 'D', 'A', 'C', 'B'}
Closure of ['D', 'E']: {'E', 'C', 'D'}
Closure of ['D', 'G']: {'G', 'D', 'A', 'B'}
Closure of ['E', 'G']: {'G', 'D', 'A', 'C', 'E', 'B'}
Closure of ['A', 'B', 'C']: {'G', 'D', 'A', 'C', 'B'}
Closure of ['A', 'B', 'D']: {'G', 'D', 'A', 'B'}
Closure of 